In [1]:
%%writefile BusStatistics.py

from mrjob.job import MRJob
from mrjob.protocol import TextValueProtocol
import numpy as np

class BusStatistics(MRJob):
    OUTPUT_PROTOCOL = TextValueProtocol # https://pythonhosted.org/mrjob/protocols.html#mrjob.protocol.TextValueProtocol
    
    def mapper(self, key, line):
        parts = line.split(",")
       
        date = parts[0][8:10] # Clean the string by slicing.
        time = parts[0][11:13] # Clean the string by slicing.
        lineref = parts[1]
        speed = parts[14]

        if date == "25" and time.isdigit(): # Drop the few lines that are from the previous day and the header.
            hour = int(time)
            if hour < 9:
                time = f"0{hour}:00-0{hour+1}:00"
            elif hour == 9:
                time = f"0{hour}:00-{hour+1}:00"
            else:
                time = f"{hour}:00-{hour+1}:00"
                
            yield (lineref + ";" + time), speed
        
    def reducer(self, key, values):
        # This way we get the line, hours and speed to output
        speeds = list(values)
        speeds = list(map(float, speeds)) # Conver to float so numpy works.
        line, interval = key.split(";")
        # Use round() to round mean and variance.
        yield None, f"{line};{interval};{min(speeds)};{max(speeds)};{np.mean(speeds).round()};{np.var(speeds).round()}"
        

if __name__ == '__main__':
    BusStatistics.run()

Overwriting BusStatistics.py


In [4]:
%run BusStatistics.py journeys*.csv

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs specified for inline runner
No configs specified for inline runner
No configs specified for inline runner


OSError: Input path journeys*.csv does not exist!